In [1]:
import os
import numpy as np
import xarray as xr
from functools import partial
import matplotlib.pyplot as plt
import sys
import glob

In [2]:
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['CRDS_PATH'] = '/YOUR/CRDS/PATH/crds_cache/'

In [3]:
from jwst import datamodels
from jwst.pipeline import calwebb_spec2
import jwst.assign_wcs.nirspec
from crds import get_default_context
print('Using crds context={}'.format(get_default_context()))

from exotic_jedi.stage_2 import GainStep, ReadNoiseStep, FlatFieldStep, \
    WavelengthMapStep, IntegrationTimesStep, StitchChunksStep, \
    InspectDQFlagsStep, CleanOutliersStep, DestripingRateimagesStep, \
    Extract1DBoxStep, Extract1DOptimalStep, AlignSpectraStep

Using crds context=jwst_1063.pmap


Point to where stage 1 fits files are

In [4]:
# Data and version config.
data_dir = '/your_directory/PLANET_VISIT/reduction_v1/stage_1/'

data_chunks = sorted(glob.glob(data_dir+'*_nrs1_stage_1.fits'))

print(data_chunks)

['/home/ym20900/compass/TOI_836_02_v2/reduction_v1/stage_1/jw02512010001_04102_00001-seg001_nrs1_stage_1.fits', '/home/ym20900/compass/TOI_836_02_v2/reduction_v1/stage_1/jw02512010001_04102_00001-seg002_nrs1_stage_1.fits', '/home/ym20900/compass/TOI_836_02_v2/reduction_v1/stage_1/jw02512010001_04102_00001-seg003_nrs1_stage_1.fits', '/home/ym20900/compass/TOI_836_02_v2/reduction_v1/stage_1/jw02512010001_04102_00001-seg004_nrs1_stage_1.fits']


Grab all the bits of names of files

In [5]:
data_chunk_names=[]
for file in range(len(data_chunks)):
    file_name = data_chunks[file].split('/')[-1].split('_stage_1.fit')[0]
    data_chunk_names.append(file_name)
#print(data_chunk_names)

first_name = data_chunks[0].split('/')[-1].split('_stage_1.fit')[0].split('-')[0]
data_name = '_'.join([first_name, 'nrs1'])
print(data_name)

jw02512010001_04102_00001_nrs1


Point to where we want everything to get saved to

In [6]:
reduction_dir = '/your_directory/PLANET_VISIT/'
version_dir = os.path.join(reduction_dir, 'reduction_v1')
stage_1_dir = os.path.join(version_dir, 'stage_1')
stage_2_dir = os.path.join(version_dir, 'stage_2')
for _dir in [stage_2_dir, ]:
    if not os.path.exists(_dir):
        os.mkdir(_dir)

Get all the steps set up with the setup step thingy (honestly I have no fucking clue what this does but according to the code gods it is necessary and it is important)

In [7]:
# Instantiate STScI steps for NIRSpec stage 2.
stsci_assign_wcs = calwebb_spec2.assign_wcs_step.AssignWcsStep()
stsci_extract_2d = calwebb_spec2.extract_2d_step.Extract2dStep()
stsci_srctype = calwebb_spec2.srctype_step.SourceTypeStep()
stsci_wavecorr = calwebb_spec2.wavecorr_step.WavecorrStep()

# Mod extract_2d trimming by updating wcs_step slit info.
jwst.assign_wcs.nirspec.nrs_wcs_set_input = partial(
    jwst.assign_wcs.nirspec.nrs_wcs_set_input,
    wavelength_range=[2.3e-06, 5.3e-06], slit_y_low=-1, slit_y_high=50)

# Instantiate Custom steps for NIRSpec stage 2.
custom_gain = GainStep()
custom_readnoise = ReadNoiseStep()
custom_flat = FlatFieldStep()
custom_wavelength_map = WavelengthMapStep()
custom_integration_times = IntegrationTimesStep()
stitch_chunks = StitchChunksStep()
inspect_dq_flags = InspectDQFlagsStep()
clean_outliers = CleanOutliersStep()
destripe_rateimages = DestripingRateimagesStep()
extract_1d_box = Extract1DBoxStep()
extract_1d_optimal = Extract1DOptimalStep()
align_spectra = AlignSpectraStep()

2023-03-09 13:34:06,108 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.
2023-03-09 13:34:06,113 - stpipe.Extract2dStep - INFO - Extract2dStep instance created.
2023-03-09 13:34:06,119 - stpipe.SourceTypeStep - INFO - SourceTypeStep instance created.
2023-03-09 13:34:06,123 - stpipe.WavecorrStep - INFO - WavecorrStep instance created.
2023-03-09 13:34:06,128 - stpipe.GainStep - INFO - GainStep instance created.
2023-03-09 13:34:06,131 - stpipe.ReadNoiseStep - INFO - ReadNoiseStep instance created.
2023-03-09 13:34:06,135 - stpipe.FlatFieldStep - INFO - FlatFieldStep instance created.
2023-03-09 13:34:06,138 - stpipe.WavelengthMapStep - INFO - WavelengthMapStep instance created.
2023-03-09 13:34:06,141 - stpipe.IntegrationTimesStep - INFO - IntegrationTimesStep instance created.
2023-03-09 13:34:06,144 - stpipe.StitchChunksStep - INFO - StitchChunksStep instance created.
2023-03-09 13:34:06,146 - stpipe.InspectDQFlagsStep - INFO - InspectDQFlagsStep instance created.
2023

In [8]:
# Read in any chunk.
stage_1_any_data_chunk = os.path.join(
    stage_1_dir, '{}_stage_1.fits'.format(data_chunk_names[0]))
dm_any_stage_1 = datamodels.CubeModel(stage_1_any_data_chunk)

### Auxiliary Data

In [9]:
# Stage 2 reduction, part 1: auxiliary data.
proc = stsci_assign_wcs.call(dm_any_stage_1)
proc = stsci_extract_2d.call(proc)
proc = stsci_srctype.call(proc)
proc = stsci_wavecorr.call(proc)
gain = custom_gain.call(
    proc, data_base_name=data_name, data_chunk_name=data_chunk_names[0],
    stage_1_dir=stage_1_dir, stage_2_dir=stage_2_dir,
    trim_col_start=5, trim_col_end=-5, median_value=True)
readnoise = custom_readnoise.call(
    proc, gain_value=gain, data_base_name=data_name,
    data_chunk_name=data_chunk_names[0],
    stage_1_dir=stage_1_dir, stage_2_dir=stage_2_dir,
    trim_col_start=5, trim_col_end=-5, median_value=True)
flat = custom_flat.call(
    proc, data_base_name=data_name, data_chunk_name=data_chunk_names[0],
    stage_2_dir=stage_2_dir, trim_col_start=5, trim_col_end=-5,
    apply=False, skip=True)
wavelength_map = custom_wavelength_map.call(
    proc, data_base_name=data_name, stage_2_dir=stage_2_dir,
    trim_col_start=5, trim_col_end=-5)
integration_times, int_time_s = custom_integration_times.call(
    data_chunk_names, data_base_name=data_name,
    stage_1_dir=stage_1_dir, stage_2_dir=stage_2_dir)

2023-03-09 13:34:22,417 - CRDS - INFO -  Fetching  /home/ym20900/crds_cache/mappings/jwst/jwst_nirspec_ote_0028.rmap    1.2 K bytes  (1 / 3 files) (0 / 6.9 K bytes)
2023-03-09 13:34:22,735 - CRDS - INFO -  Fetching  /home/ym20900/crds_cache/mappings/jwst/jwst_nirspec_0317.imap    5.2 K bytes  (2 / 3 files) (1.2 K / 6.9 K bytes)
2023-03-09 13:34:23,057 - CRDS - INFO -  Fetching  /home/ym20900/crds_cache/mappings/jwst/jwst_1063.pmap        580 bytes  (3 / 3 files) (6.4 K / 6.9 K bytes)
2023-03-09 13:34:24,136 - stpipe - CRITICAL - config: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': None, 'sip_approx': True, 'sip_max_pix_error': 0.25, 'sip_degree': None, 'sip_max_inv_pix_error': 0.25, 'sip_inv_degree': None, 'sip_npoints': 32, 'slit_y_low': -0.55, 'slit_y_high': 0.55, 'override_distortion

2023-03-09 13:34:36,472 - stpipe.Extract2dStep - INFO - Results used CRDS context: jwst_1063.pmap
2023-03-09 13:34:36,473 - stpipe.Extract2dStep - INFO - Step Extract2dStep done
2023-03-09 13:34:37,864 - stpipe - CRITICAL - config: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': None, 'source_type': None}


2023-03-09 13:34:37,869 - stpipe.SourceTypeStep - INFO - SourceTypeStep instance created.
2023-03-09 13:34:38,220 - stpipe.SourceTypeStep - INFO - Step SourceTypeStep running with args (<SlitModel(1524, 32, 2048) from jw02512010001_04102_00001-seg001_nrs1_stage_1.fits>,).
2023-03-09 13:34:38,221 - stpipe.SourceTypeStep - INFO - Step SourceTypeStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False

2023-03-09 13:34:48,841 - stpipe.FlatFieldStep - INFO - Step FlatFieldStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'data_base_name': 'jw02512010001_04102_00001_nrs1', 'data_chunk_name': 'jw02512010001_04102_00001-seg001_nrs1', 'stage_2_dir': '/home/ym20900/compass/TOI_836_02_v2/reduction_v1/stage_2', 'trim_col_start': 5, 'trim_col_end': -5, 'apply': False}
2023-03-09 13:34:48,842 - stpipe.FlatFieldStep - INFO - Step skipped.
2023-03-09 13:34:48,845 - stpipe.FlatFieldStep - INFO - Step FlatFieldStep done
2023-03-09 13:34:49,054 - CRDS - ERROR -  Error determining best reference for 'pars-wavelengthmapstep'  =   Unknown reference type 'pars-wavelengthmapstep'
2023-03-09 13:34:49,057 - stpipe - CRITICAL - config: {'data_base_name': 'jw02512010001_04102_00001_nrs1', 